In [5]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt

In [117]:
imdb = keras.datasets.imdb
# num_words는 몇 개의 단어를 사용할 것인지 지정한다.
# 훈련 데이터에서 가장 많이 등장하는 상위 10,000개의 단어를 선택한다.
(x_train, y_train) , (x_test, y_test) = imdb.load_data(num_words=10000)

In [118]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(25000,)
(25000,)
(25000,)
(25000,)


In [121]:
# 정수들이 출력되는 것을 볼 수 있는데,이것은 단어 임베딩이 아니다.
# 임베딩은 부동소수점수로 된 벡터라는 것을 잊지 말자
# imdb 데이터들은 미리 전처리가 되어 있다.
# 즉 단어들을 토큰화하고 이것을 정수 인덱스로 인코딩한 상태이다.
# 정수 인덱스는 각 단어들의 출현 회수에 따라 결정된다.
# 즉 더 많이 등장하는 단어가 낮은 인덱스가 된다.
print(x_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [122]:
len(x_train[0]), len(x_train[1])

(218, 189)

In [123]:
y_train[0], y_train[1]

(1, 0)

In [124]:
np.unique(y_train,return_counts=True)

(array([0, 1], dtype=int64), array([12500, 12500], dtype=int64))

In [125]:
# 단어 -> 정수 인덱스 딕셔너리
word_to_index = imdb.get_word_index()

# 처음 몇 개의 인덱스는 특수 용도로 사용된다.
## 딕셔너리의 모든 키값에 +3 씩한다
word_to_index = {k:(v+3) for k,v in word_to_index.items()}
## 몇몇개의 특수한 용도로 쓰이는 것을 추가한다.
word_to_index["<PAD>"] = 0
word_to_index["<START>"] = 1
word_to_index["<UNK>"] = 2
word_to_index["<UNUSED>"] = 3

# 딕셔너리의 아이탬을 서로 바꿔준다.
index_to_word = dict([(value, key) for (key, value) in word_to_index.items()])

In [126]:
# join 연산을 통해 리스트에 있는 항목들을 다 붙여준다.
print(' '.join([index_to_word[index] for index in x_train[0]]))

<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wha

In [127]:
# 전처리
## 정수 배열로 되어 있는 것을 부동소수점수를 가지고 있는 텐서로 변환되어야 한다.
## 우리가 학습했다시피 변환하는 방법에는 몇 가지가 잇는데
## 원-핫 코딩 : 정수 배열을 0과 1로 이루어진 벡터로 변환한다.
## 예를 들어 배열 [3, 7]을 인덱스 3과 7만 1이고 나머지는 모두 0 인 10,000 차원 벡터로 변환할 수 있다.
## 입력층으로는 실수 벡터 데이터를 다룰 수 있는 Dense 층을 신경망의 첫 번째 레이어로 사용한다
## 이 방법은 num_words * num_reviews 크기의 행렬이 필요하기 때문에 메모리를 많이 사용한다.
## 임베딩 방법 : 정수 배열이 길이가 모두 같도록 패딩(padding)을 추가해 max_length * (embedding) 레이어를
## 신경망의 첫 번째 레이어로 사용할 수 있다. 임베딩 레이어에서 정수 배열을 부동소수점 벡터로 변환한다.

from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import *

In [128]:
x_train = pad_sequences(x_train, maxlen=100)
x_test = pad_sequences(x_test, maxlen=100)

In [129]:
len(x_train[0]), len(x_train[1])

(100, 100)

In [130]:
print(x_train[0])

[1415   33    6   22   12  215   28   77   52    5   14  407   16   82
    2    8    4  107  117 5952   15  256    4    2    7 3766    5  723
   36   71   43  530  476   26  400  317   46    7    4    2 1029   13
  104   88    4  381   15  297   98   32 2071   56   26  141    6  194
 7486   18    4  226   22   21  134  476   26  480    5  144   30 5535
   18   51   36   28  224   92   25  104    4  226   65   16   38 1334
   88   12   16  283    5   16 4472  113  103   32   15   16 5345   19
  178   32]


In [33]:
# 신경망 구축
vocab_size = 10000

model = Sequential()
# Embedding 층은 인코딩된 단어를 입력받고 각 단어 인덱스에 해당하는 임베딩 벡터를 찾는다.
# 첫 번째 인수는 단어 집합의 크기로 여기서는 10000으로 하였다. 두 번째 인수는 임베딩 후의 벡터크기로서
# 64으로 설정하였다. 따라서 리뷰 데이터에서 모든 단어는 64차원의 임베딩 벡터로 표현된다. 이 벡터는 모델이
# 훈련되면서 학습된다. 리뷰의 최대 길이를 100으로 제한하였다.
model.add(Embedding(vocab_size, 64, input_length=100))
# 이 고정 길이의 출력 벡터는 64개의 은닉 노드를 가진 FC 층을 거친다.
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
# 마지막 층은 이진 분류이므로 뉴런 하나로 되어 있고 시그모이드 활성화 함수를 사용하여 0과 1 사이의 실수를 출력한다.
# 이 값은 확률 또는 신뢰도를 나타낸다.
model.add(Dense(1, activation='sigmoid'))

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 64)           640000    
                                                                 
 flatten (Flatten)           (None, 6400)              0         
                                                                 
 dense (Dense)               (None, 64)                409664    
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,049,729
Trainable params: 1,049,729
Non-trainable params: 0
_________________________________________________________________


In [39]:
model.compile(loss='binary_crossentropy', optimizer='adam',
              metrics=['accuracy'])

In [40]:
history = model.fit(x_train, y_train, batch_size=64, epochs=20, verbose=1,
                    validation_data=(x_test,y_test))

Epoch 1/20
391/391 [==============================] - 5s 6ms/step - loss: 0.4575 - accuracy: 0.7672 - val_loss: 0.3340 - val_accuracy: 0.8515
Epoch 2/20
391/391 [==============================] - 2s 5ms/step - loss: 0.1673 - accuracy: 0.9398 - val_loss: 0.3970 - val_accuracy: 0.8358
Epoch 3/20
391/391 [==============================] - 2s 5ms/step - loss: 0.0247 - accuracy: 0.9945 - val_loss: 0.5329 - val_accuracy: 0.8344
Epoch 4/20
391/391 [==============================] - 2s 5ms/step - loss: 0.0045 - accuracy: 0.9998 - val_loss: 0.6012 - val_accuracy: 0.8394
Epoch 5/20
391/391 [==============================] - 2s 5ms/step - loss: 0.0023 - accuracy: 0.9997 - val_loss: 0.6514 - val_accuracy: 0.8379
Epoch 6/20
391/391 [==============================] - 2s 5ms/step - loss: 8.4364e-04 - accuracy: 1.0000 - val_loss: 0.6952 - val_accuracy: 0.8379
Epoch 7/20
391/391 [==============================] - 2s 5ms/step - loss: 5.7167e-04 - accuracy: 1.0000 - val_loss: 0.7241 - val_accuracy: 0.840

In [42]:
result = model.evaluate(x_test, y_test, verbose=2)
print(result)

782/782 - 2s - loss: 0.9841 - accuracy: 0.8115 - 2s/epoch - 2ms/step
[0.9840518236160278, 0.811519980430603]


In [43]:
review = 'Aside from the super obvious plot holes and very poor story overall, the dragged-out unnecessary dialogue made this film unbearable and extremely boring. The way too long 1h 39min film length felt like 4 hours and I found myself saying "get on with it already, who cares!" when the two leads would just ramble on about nothing relevant. This movie may have been interesting if it was a 30 min short filmle and extremely boring. The way too long 1h 39min film length felt like 4 hours and I found myself saying "get on with it already, who cares!" when the two leads would just ramble on about nothing relevant. This movie may have been interesting if it was a 30 min short film (which oddly enough is the only minimal writing experience Adam Gaines has'

import re
review = re.sub("[^0-9a-zA-Z]","", review).lower()

In [44]:
review_encoding = []
for w in review.split():
    index = word_to_index.get(w, 2)
    if index <= 10000:
        review_encoding.append(index)
    else:
        review_encoding.apeend(word_to_index["UNK"])
        
# 2차원 리스트로 전달하여야 한다.
test_input = pad_sequences([review_encoding], maxlen=100)
value = model.predict(test_input)
if(value>0.5):
    print('긍정적인 리뷰입니다.')
else:
    print('부정적인 리뷰입니다.')

부정적인 리뷰입니다.


In [133]:
word_to_index.get('the')

4

In [109]:
from tensorflow.keras.layers import LSTM
lstm_model = Sequential()
lstm_model.add(Embedding(vocab_size, 64, input_length=100))
lstm_model.add(LSTM(16))
lstm_model.add(Dense(64, activation='relu'))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(1, activation='sigmoid'))

lstm_model.summary()

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    (None, 100, 64)           640000    
                                                                 
 lstm_21 (LSTM)              (None, 16)                5184      
                                                                 
 dense_29 (Dense)            (None, 64)                1088      
                                                                 
 dropout_10 (Dropout)        (None, 64)                0         
                                                                 
 dense_30 (Dense)            (None, 1)                 65        
                                                                 
Total params: 646,337
Trainable params: 646,337
Non-trainable params: 0
_________________________________________________________________


In [110]:
lstm_model.compile( loss='binary_crossentropy', optimizer='adam',
              metrics=['accuracy'])

In [111]:
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)

history = lstm_model.fit(x_train, y_train, batch_size=64, epochs=20, verbose=1,
                    validation_data=(x_test,y_test),callbacks=[earlystop])

Epoch 1/20
391/391 [==============================] - 8s 15ms/step - loss: 0.4527 - accuracy: 0.7795 - val_loss: 0.3502 - val_accuracy: 0.8498
Epoch 2/20
391/391 [==============================] - 5s 13ms/step - loss: 0.2615 - accuracy: 0.8974 - val_loss: 0.3696 - val_accuracy: 0.8345
Epoch 3/20
391/391 [==============================] - 5s 13ms/step - loss: 0.1834 - accuracy: 0.9316 - val_loss: 0.4233 - val_accuracy: 0.8404
Epoch 4/20
391/391 [==============================] - 5s 13ms/step - loss: 0.1338 - accuracy: 0.9528 - val_loss: 0.5065 - val_accuracy: 0.8347
Epoch 5/20
391/391 [==============================] - 5s 13ms/step - loss: 0.0965 - accuracy: 0.9656 - val_loss: 0.5720 - val_accuracy: 0.8334
Epoch 6/20
391/391 [==============================] - 5s 13ms/step - loss: 0.0754 - accuracy: 0.9741 - val_loss: 0.6280 - val_accuracy: 0.8270


In [134]:
result = lstm_model.evaluate(x_test, y_test, verbose=2)
print(result)

782/782 - 3s - loss: 0.6280 - accuracy: 0.8270 - 3s/epoch - 4ms/step
[0.6280211806297302, 0.8270000219345093]
